In [1]:
import kenlm
from tqdm import tqdm
import fastText

modellm = kenlm.Model('kenlmmodel/yelp.arpa')

classifier_model = fastText.load_model('fasttextmodel/model_yelp.bin')

In [2]:
print(modellm.perplexity('I just hate the food .'))

396.7080016573861


In [6]:
texts = ['worst food ever', 'very bad nice place']
print(classifier_model.predict(texts))

([['__label__0'], ['__label__1']], array([[1.00001001],
       [0.97183311]]))


In [5]:
path_to_txt_file = './ref_files/ref1_only_predictions.txt'
label = '__label__0'

with open(path_to_txt_file) as f:
    sentences = f.read().splitlines()
    #content = [l.decode('utf8', 'ignore') for l in f.readlines()]
# you may also want to remove whitespace characters like `\n` at the end of each line
# sentences = [x.strip() for x in sentences]
labels = classifier_model.predict(sentences)

correct_pred = 0
perplexity_sum = 0

with open(path_to_txt_file+'.log',"w") as text_file:
    for _, sentence in tqdm(enumerate(sentences)):
        #pred_label = learn_classifier.predict(sentence)[1].item()
        pred_perplexity = modellm.perplexity(sentence)
        perplexity_sum += pred_perplexity
        if(labels[0][_][0]==label):
            correct_pred+=1
        text_file.write(sentence+'\t'+labels[0][_][0]+'\t'+str(round(pred_perplexity, 2))+'\n')

with open(path_to_txt_file+'.log', 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        accuracy = correct_pred/len(sentences)
        perplexity_avg = round(perplexity_sum/len(sentences), 2)
        print('Accuracy: {} \nAverage Perplexity: {}\n'.format(accuracy, perplexity_avg))
        f.write('Accuracy: {} \nAverage Perplexity: {}\n'.format(accuracy, perplexity_avg))
        f.write('--------------------------------------------------------------------------------------------- \n')

500it [00:00, 87798.38it/s]

Accuracy: 0.88 
Average Perplexity: 176.07



In [37]:
#creating text files for glue

import pandas as pd

df = pd.read_csv('amazon_all_model_prediction_1.csv', header = None)

list_sentences = df[1:len(df)].values.tolist()

In [33]:
list_sentences[0]

['the group of hikers is resting in front of a mountain .',
 'the man is standing on the grass , trying to get the ball .',
 'a skateboarder are is wearing rollerblades is in a crowd .',
 'the large crowd is in front of a rock .',
 'the drunk skier is stupidly stupidly in front of a mountain .',
 'the group of boys trying to be crash in front of a mountain .',
 'the group of hikers is resting in front of a mountain looking for bigfoot .',
 'the group of hikers is resting in front of a mountain looking for bigfoot .',
 'several hikers , gear , mountain in hopes of seeing a bear .']

In [39]:
id = 1
for i in range(0, len(list_sentences[0])):
    with open('./gleu_res/ref1/amazon'+str(id)+".txt", "w") as text_file:
        for j in range(0, len(list_sentences)):
            if(pd.isnull(list_sentences[j][i])):
                continue
            text_file.write(list_sentences[j][i]+"\n")
    id+=1

In [69]:
import pandas as pd
import numpy as np

df0 = pd.read_csv('matrics_yelp_all_model_prediction_0.csv', header = None)
df1 = pd.read_csv('matrics_yelp_all_model_prediction_1.csv', header = None)

In [33]:
list_cells = df0[1:].values.tolist()

In [97]:
list_newval =[]
for j in range(1,12):
    temp_list=[]
    for i in range(2,9):
        temp_list.append(str(round(((float(df0[i][j])+float(df1[i][j]))/2.),4 )))
    list_newval.append(temp_list)

In [101]:
models_list = ['Source',
  'CROSSALIGNED',
  'STYLEEMBEDDING',
  'MULTIDECODER',
  'RETRIEVEONLY',
  'TEMPLATEBASED',
  'DELETEONLY',
  'DELETEANDRETRIEVE',
  'BERT_DEL',
  'BERT_RET',
  'HUMAN']

df_res = pd.DataFrame(list_newval, columns=['GLEU', 'BLEU_source','BLEU_human','fasttext_classifier','klm_ppl','fastai_classifier','fastailm_ppl'])
df_res.insert(loc=0, column='model', value=models_list)


In [102]:
df_res

,model,GLEU,BLEU_source,BLEU_human,fasttext_classifier,klm_ppl,fastai_classifier,fastailm_ppl
0,Source,0.0757,1.0,0.5399,0.045,73.0582,0.02,32.9232
1,CROSSALIGNED,0.0438,0.4561,0.3105,0.768,70.7022,0.737,165.8417
2,STYLEEMBEDDING,0.0587,0.7894,0.4378,0.095,105.7637,0.078,330.7297
3,MULTIDECODER,0.0503,0.5601,0.348,0.506,159.1549,0.482,607.7182
4,RETRIEVEONLY,0.0104,0.2282,0.1798,0.921,8.2492,0.923,52.2676
5,TEMPLATEBASED,0.0907,0.6091,0.4039,0.824,2085.4696,0.803,1451.1379
6,DELETEONLY,0.0644,0.54,0.3663,0.859,100.1707,0.853,205.4436
7,DELETEANDRETRIEVE,0.0686,0.5354,0.3655,0.878,118.7094,0.888,267.4751
8,BERT_DEL,0.1123,0.6828,0.4658,0.708,96.6564,0.728,89.459
9,BERT_RET,0.1009,0.656,0.4456,0.555,135.7993,0.582,148.1388


In [103]:
df_res.to_csv('matrics_average.csv')

In [8]:
import numpy as np

In [10]:
mat1 = np.array([
0.076314,
0.044249,
0.059500,
0.047856,
0.008023,
0.001988,
0.063792,
0.071165,
0.122037,
0.125641,
0.100860,
0.092407,
0.110423,
0.110237,
0.107167,
0.109255,
0.104249,
0.109463,
0.015993,
0.040456,
0.000000,
0.000000,
0.102796,
0.015269,
1.000000
])

In [9]:
mat0 = np.array([
0.075047,
0.043375,
0.057970,
0.052730,
0.012828,
0.082611,
0.065006,
0.066123,
0.109153,
0.093038,
0.100581,
0.084421,
0.114485,
0.090462,
0.104663,
0.093399,
0.102966,
0.094599,
0.019721,
0.034744,
0.004677,
0.000000,
0.096695,
0.012745,
1.000000
])

In [11]:
matavg = (mat0+mat1)/2

In [14]:
[print(i) for i in mat1]

0.076314
0.044249
0.0595
0.047856
0.008023
0.001988
0.063792
0.071165
0.122037
0.125641
0.10086
0.092407
0.110423
0.110237
0.107167
0.109255
0.104249
0.109463
0.015993
0.040456
0.0
0.0
0.102796
0.015269
1.0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [18]:
dictofppl = {'Source': 179.98222876548766,
'CROSSALIGNED': 368.56972133636475,
'STYLEEMBEDDING': 449.12290202522274,
'MULTIDECODER': 730.3358956260681,
'RETRIEVEONLY': 208.2480753135681,
'TEMPLATEBASED': 385.7471610374451,
'DELETEONLY': 336.44651431274417,
'DELETEANDRETRIEVE': 425.2543611755371,
'BERT_DEL': 223.99889970397948,
'SEL_DEL': 201.02439992332458,
'BERT_RET_USE': 301.5724952011108,
'SAL_RET_USE': 256.1212079248428,
'BERT_RET_TFIDF': 313.81375119018554,
'SAL_RET_TFIDF': 303.5601326875686,
'BERT_RET_GLOVE': 353.2743957939148,
'SAL_RET_GLOVE': 307.8996427268982,
'BERT_RET_RANDOM': 415.94113478660586,
'SAL_RET_RANDOM': 265.90657262611387,
'RET_ONLY_USE': 188.06904032325744,
'RET_ONLY_TFIDF': 269.54081302833555,
'RET_ONLY_GLOVE': 310.337988117218,
'RET_ONLY_RANDOM': 227.66278438282012,
'HUMAN': 140.11555947446823}

In [19]:
[print(dictofppl[i]) for i in dictofppl.keys()]

179.98222876548766
368.56972133636475
449.12290202522274
730.3358956260681
208.2480753135681
385.7471610374451
336.44651431274417
425.2543611755371
223.99889970397948
201.02439992332458
301.5724952011108
256.1212079248428
313.81375119018554
303.5601326875686
353.2743957939148
307.8996427268982
415.94113478660586
265.90657262611387
188.06904032325744
269.54081302833555
310.337988117218
227.66278438282012
140.11555947446823


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]